In [1]:
import pickle
import time

import numpy as np
import pandas as pd

In [2]:
from sknetwork.data import from_edge_list
import networkx as nx

In [3]:
# DNp01 (giant fiber) to DNp11
body_ids = ["2307027729","5813024015", "1565846637", "1405231475", "1466998977", "5813023322", "1100404581", "1226887763", "1228264951", "512851433", "5813026936", "1281324958"]
DNp_ids = [int(i) for i in body_ids]

In [15]:
DIR = "./data/consensusResults2"

In [16]:
consensusResults = np.load(f"{DIR}/consensusResults.npy", allow_pickle=True)

In [9]:
# adj_mat = np.loadtxt("adj_mat.csv", delimiter=",")
all_connection_df = pd.read_csv("./data/all_connection_df.csv")
dfFilt = all_connection_df[['bodyId_pre', 'bodyId_post', 'weight']] # sknetwork uses 3rd col as weight

In [10]:
# sknetwork: only needed bc Louvain's clustering assignment output is based on graph.names order
graph = from_edge_list(list(dfFilt.itertuples(index=False)), weighted=True, directed=True) # without directed=True, wrong # of elements

In [11]:
graph.adjacency, graph.names

(<5596x5596 sparse matrix of type '<class 'numpy.int64'>'
 	with 389065 stored elements in Compressed Sparse Row format>,
 array([ 326253554,  357245785,  357249472, ..., 7112615304, 7112622763,
        7112624834]))

In [12]:
# create directed, weighted networkx graph from dataframe of edges & weights
# NOTE: it seems that NetworkX conversion shuffles edge ordering (so clustering results may be different)
G = nx.from_pandas_edgelist(all_connection_df, 'bodyId_pre', 'bodyId_post', 'weight', create_using=nx.DiGraph())

In [13]:
G.number_of_nodes(), G.number_of_edges()

(5596, 389065)

In [11]:
'''
# cluster_betweenness = nx.betweenness_centrality(cluster_subgraph, weight='weight') # maybe set endpoints=True
# cluster_degreeCentrality = nx.degree_centrality(cluster_subgraph)
# cluster_eigenCentrality = nx.eigenvector_centrality(cluster_subgraph, weight='weight')
# degAssortCoeff = nx.degree_assortativity_coefficient(cluster_subgraph, weight='weight')
# avgDegCon = nx.average_degree_connectivity(cluster_subgraph, weight='weight')

# groupBetCentrality = nx.group_betweenness_centrality(G, cluster_nodes, weight='weight')
# groupDegreeCentrality = nx.group_degree_centrality(G, cluster_nodes)

# smallWorld = nx.algorithms.smallworld.sigma(cluster_subgraph.to_undirected())

# cluster_modularity = nx.community.modularity(G, cluster_nodes, weight='weight')
'''

"\n# cluster_betweenness = nx.betweenness_centrality(cluster_subgraph, weight='weight') # maybe set endpoints=True\n# cluster_degreeCentrality = nx.degree_centrality(cluster_subgraph)\n# cluster_eigenCentrality = nx.eigenvector_centrality(cluster_subgraph, weight='weight')\n# degAssortCoeff = nx.degree_assortativity_coefficient(cluster_subgraph, weight='weight')\n# avgDegCon = nx.average_degree_connectivity(cluster_subgraph, weight='weight')\n\n# groupBetCentrality = nx.group_betweenness_centrality(G, cluster_nodes, weight='weight')\n# groupDegreeCentrality = nx.group_degree_centrality(G, cluster_nodes)\n\n# smallWorld = nx.algorithms.smallworld.sigma(cluster_subgraph.to_undirected())\n\n# cluster_modularity = nx.community.modularity(G, cluster_nodes, weight='weight')\n"

In [14]:
# for each consensus result, for each cluster, extract its subgraph
# calculate graph theory metrics (degree centrality, betweenness centrality, small worldness) for each subgraph
# maybe calculate modularity (reflects degree of community structure) and assortativity (reflects tendency of nodes to connect to similar nodes)

# data structure to store results: dictionary of list containing a dictionary of networkx's values
'''
{
    consensusIterationNum: [
        {cluster1's metric}, 
        {cluster2's metric}, 
        ...
        ],
     consensusIterationNum: [
        {cluster1's metric}, 
        {cluster2's metric}, 
        ...
        ]
}
'''

startTime = time.time()

metricName = "degreeCentrality" # CHANGE THIS
graphMetric = {}

print(f"Calculating graph theory metrics: {metricName}\n")
for result in consensusResults:
    iteration, consensus = result
    graphMetric[iteration] = []
    
    iterStartTime = time.time()
    for clusterId in range(1, consensus.max()+1): # cluster indices are beween 1-n (inclusive)
        cluster_indices = np.where(consensus == clusterId)[0]
        cluster_nodes = [graph.names[i] for i in cluster_indices] # bodyIds
        cluster_subgraph = G.subgraph(cluster_nodes)
        # print(cluster_subgraph.number_of_nodes(), cluster_subgraph.number_of_edges())

        ### calculate graph theory metrics - uncomment as needed ###
        # Degree Centrality
        cluster_degreeCentrality = nx.degree_centrality(cluster_subgraph)
        graphMetric[iteration].append(cluster_degreeCentrality)

        # Betweenness Centrality
        # cluster_betweenness = nx.betweenness_centrality(cluster_subgraph, weight='weight') # maybe set endpoints=True
        # graphMetric[iteration].append(cluster_betweenness)

        # Small Worldness (sigma)
        # smallWorld = nx.algorithms.smallworld.sigma(cluster_subgraph.to_undirected())
        # graphMetric[iteration].append(smallWorld)

        # Eigenvector Centrality
        # cluster_eigenvector = nx.eigenvector_centrality_numpy(cluster_subgraph, weight='weight') # numpy version is faster for larger graphs
        # graphMetric[iteration].append(cluster_eigenvector)

        # VoteRank
        # if clusterId == 4:
        #     cluster_voterank = nx.voterank(cluster_subgraph)
    print(f"Iteration {iteration} runtime: {(time.time()-iterStartTime):.2f}s")
    # print()

print(f"Total runtime: {(time.time()-startTime):.2f}s")

Calculating graph theory metrics: degreeCentrality

Iteration 10 runtime: 0.55s
Iteration 25 runtime: 0.44s
Iteration 50 runtime: 0.43s
Iteration 75 runtime: 0.43s
Iteration 100 runtime: 0.43s
Iteration 150 runtime: 0.48s
Iteration 200 runtime: 0.51s
Iteration 250 runtime: 0.43s
Iteration 300 runtime: 0.43s
Iteration 350 runtime: 0.43s
Iteration 400 runtime: 0.42s
Iteration 450 runtime: 0.42s
Iteration 500 runtime: 0.43s
Total runtime: 5.82s


In [18]:
# save graph theory results
with open(f'{DIR}/{metricName}.pkl', 'wb') as handle:
    pickle.dump(graphMetric, handle)

In [19]:
# read results
with open(f'{DIR}/{metricName}.pkl', 'rb') as handle:
    temp = pickle.load(handle)

In [31]:
sum(cluster_degreeCentrality.values()) / len(cluster_degreeCentrality)

0.07173788325575241

In [74]:
groupDegreeCentrality

0.482007996446024

In [65]:
sum(avgDegCon.values())/len(avgDegCon.values()) # doesn't seem meaningful

145.67808078312498

In [64]:
degAssortCoeff

-0.0840938857573405

In [50]:
sum(cluster_eigenCentrality.values()) / len(cluster_eigenCentrality)

0.008290702240462593

In [53]:
sum(cluster_betweenness.values()) / len(cluster_betweenness)

0.004125238268360178

In [57]:
cluster_betweenness[2307027729], cluster_betweenness[5813024015] # DNp01,02

(0.036588253808819825, 0.045981048093354494)

In [25]:
### find subgraph consisting of nodes in cluster 1
# first extract the nodes associated with cluster 1
consensus250 = consensusResults[0][1] # consensus clustering labels of each of the 5596 nodes for iteration 250
cluster1_nodeIndices = np.where(consensus250 == 4)[0] # indices of cluster1 neurons based on graph.names
cluster1_nodes = [graph.names[i] for i in cluster1_nodeIndices] # get cluster1 neuron bodyIds from graph.names

# then extract the subgraph
cluster1_subgraph = G.subgraph(cluster1_nodes)

In [26]:
cluster1_subgraph.number_of_nodes(), cluster1_subgraph.number_of_edges()

(1094, 42890)